# Exploring Eviction Data

In [30]:
import pandas as pd
import numpy as np
from collections import Counter

import matplotlib as plt
import gmplot

In [8]:
this_file = open("/Users/edie/Box Sync/GitThings/loveSF/Eviction_Notices.csv", "r")
evictions = pd.read_csv(this_file)

evictions.head()

,Eviction ID,Address,City,State,Eviction Notice Source Zipcode,File Date,Non Payment,Breach,Nuisance,Illegal Use,...,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Constraints Date,Supervisor District,Neighborhoods - Analysis Boundaries,Location
0,M162268,2700 Block Of San Bruno Avenue,San Francisco,CA,94134,06/24/2016,False,False,False,False,...,False,False,False,False,False,False,08/22/2021,9.0,Portola,"(37.727050746607, -122.403281820714)"
1,M161957,300 Block Of Park Street,San Francisco,CA,94110,06/06/2016,False,False,False,False,...,False,False,False,False,False,False,NaN,9.0,Bernal Heights,"(37.7364374645373, -122.417989910582)"
2,M162256,200 Block Of Seneca Avenue,San Francisco,CA,94112,06/29/2016,False,False,False,False,...,False,False,False,False,False,False,08/25/2021,11.0,Outer Mission,"(37.7205429316262, -122.443264852669)"
3,M162135,200 Block Of Dolores Street,San Francisco,CA,94103,06/28/2016,False,True,False,False,...,False,False,False,False,False,False,NaN,8.0,Castro/Upper Market,"(37.7652067507312, -122.426591617441)"
4,M161901,1200 Block Of 9th Avenue,San Francisco,CA,94122,06/02/2016,False,False,False,False,...,False,False,False,False,False,False,NaN,5.0,Inner Sunset,"(37.764977785911, -122.4664456379)"


In [10]:
list(evictions)

['Eviction ID',
 'Address',
 'City',
 'State',
 'Eviction Notice Source Zipcode',
 'File Date',
 'Non Payment',
 'Breach',
 'Nuisance',
 'Illegal Use',
 'Failure to Sign Renewal',
 'Access Denial',
 'Unapproved Subtenant',
 'Owner Move In',
 'Demolition',
 'Capital Improvement',
 'Substantial Rehab',
 'Ellis Act WithDrawal',
 'Condo Conversion',
 'Roommate Same Unit',
 'Other Cause',
 'Late Payments',
 'Lead Remediation',
 'Development',
 'Good Samaritan Ends',
 'Constraints Date',
 'Supervisor District',
 'Neighborhoods - Analysis Boundaries',
 'Location']

<font color="navy">
## How many percent of evictions were due to non-payment?
Overall, 5.96%. I feel like this is inaccurate.

In [14]:
100*(len(evictions[evictions["Non Payment"]==True]) * (len(evictions["Non Payment"])**(-1)))

5.961844197138314

<font color="navy">
## Which zip code has most of the evictions?
The top 5 most evicted zip codes are: 94110, 94109, 94117, 94112, 94122.

In [21]:
Counter(evictions["Eviction Notice Source Zipcode"]).most_common()[0:5]

[('94110', 4010),
 ('94109', 2323),
 ('94117', 2302),
 ('94112', 2071),
 ('94122', 2011)]

<font color="navy">
## Are these due to non-payment?
Yes.

In [70]:
top_5 = [94110, 94109, 94117, 94112, 94122]

In [103]:
def getNeighborhoods(zip_code):
    neighborhoods = list(set(evictions[evictions["Eviction Notice Source Zipcode"]==zip_code]["Neighborhoods - Analysis Boundaries"]))
    # START AT 1 DUE TO NAN
    return(neighborhoods[1:])

In [45]:
def getNonPaidPercent(zip_code):
    df = evictions[evictions["Eviction Notice Source Zipcode"]==zip_code]
    return(100*(len(df[df["Non Payment"]==True]) * (len(df["Non Payment"])**(-1))))

In [47]:
def getLatePercent(zip_code):
    df = evictions[evictions["Eviction Notice Source Zipcode"]==zip_code]
    return(100*(len(df[df["Late Payments"]==True]) * (len(df["Late Payments"])**(-1))))

In [49]:
def getDemolition(zip_code):
    df = evictions[evictions["Eviction Notice Source Zipcode"]==zip_code]
    return(100*(len(df[df["Demolition"]==True]) * (len(df["Demolition"])**(-1))))

In [106]:
top5_df = pd.DataFrame({"zips":top_5})
top5_df["nonpaid"] = [getNonPaidPercent(x) for x in top_5]
top5_df["late"] = [getLatePercent(x) for x in top_5]
top5_df["demolition"] = [getDemolition(x) for x in top_5]
top5_df["neighborhoods"] = [", ".join(getNeighborhoods(x)) for x in top_5]

In [107]:
top5_df

,zips,nonpaid,late,demolition,neighborhoods
0,94110,7.284768,3.752759,5.298013,"Glen Park, Potrero Hill, Hayes Valley, Mission..."
1,94109,7.600000,4.000000,0.800000,"Nob Hill, Japantown, Pacific Heights, Hayes Va..."
2,94117,3.272727,3.272727,2.545455,"South of Market, Lone Mountain/USF, Hayes Vall..."
3,94112,11.111111,4.700855,5.982906,"Sunset/Parkside, West of Twin Peaks, Mission, ..."
4,94122,8.376963,3.664921,5.235602,"Nob Hill, Mission, Inner Sunset, Excelsior, Ou..."


<font color="navy">
## Can we visualize San Francisco?
Sure, can. Will do more later.

In [28]:
gmap = gmplot.GoogleMapPlotter.from_geocode("San Francisco")

In [33]:
# CREATES HTML FILE
gmap.draw("mymap.html")